In [16]:
import graphlab
import numpy as np



In [2]:
## Load the train and test data
train = graphlab.SFrame('~/Downloads/AV/train.csv')
test = graphlab.SFrame('~/Downloads/AV/test.csv')

Finished parsing file /home/santanu/Downloads/AV/train.csv

Parsing completed. Parsed 100 lines in 0.586491 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,float,float,float,float,float,float,float,float,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 546558 lines. Lines per second: 580434

Finished parsing file /home/santanu/Downloads/AV/train.csv

...

Finished parsing file /home/santanu/Downloads/AV/test.csv

Parsing completed. Parsed 100 lines in 0.107378 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/santanu/Downloads/AV/test.csv

Parsing completed. Parsed 101946 lines in 0.164579 secs.

In [4]:
## Check for class imbalance .

#print 'train record count',len(train),'test record count',len(test)
#print train.column_names()
print len(train[train['Outcome'] == 1]),len(train[train['Outcome'] == 1])/float(len(train))


318283 0.452917797361


In [5]:
##  Check to see is there are stocks ids in test which are not in train. As we can see more stocks in tests than train 
## and hence can't use stock id as a feature.

train[train['Stock_ID'] == 1]
stock_train = train['Stock_ID'].unique()
stock_test = test['Stock_ID'].unique()
print len(stock_train),len(stock_test)

stock_common =  np.intersect1d(stock_train,stock_test)
print len(stock_common),stock_common

1955 2118
1954 [   1    2    3 ..., 2197 2198 2199]


In [6]:
## Checking the test data for a stock 
test[test['Stock_ID'] == 1]

ID,timestamp,Stock_ID,Volume,Three_Day_Moving_Average,Five_Day_Moving_Average,Ten_Day_Moving_Average
1_554,554,1,0.43959,-1.09885,-1.08146,-1.02367
1_568,568,1,-0.44671,-1.06427,-1.05526,-1.07855
1_575,575,1,-0.06037,-0.70649,-0.74022,-0.83498
1_582,582,1,0.53049,-0.53751,-0.56485,-0.55882
1_589,589,1,-0.40126,-0.01625,-0.12814,-0.32254
1_596,596,1,-0.49216,-0.04059,-0.02685,-0.10138
1_603,603,1,0.91683,-0.01988,-0.02368,0.01495
1_610,610,1,-0.10582,-0.17903,-0.18882,-0.15939
1_617,617,1,-0.0831,-0.14221,-0.15555,-0.1676
1_624,624,1,-0.03765,-0.03296,-0.01134,-0.01218


In [3]:
## Feature engineering 
def std_compute(row):
    return np.std([np.int(row['Three_Day_Moving_Average']),np.int(row['Five_Day_Moving_Average']),np.int(row['Ten_Day_Moving_Average'])])
    
def create_feature(df):
    df['Three_Day_Moving_Average_na'] = df['Three_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Five_Day_Moving_Average_na'] = df['Five_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Ten_Day_Moving_Average_na'] = df['Ten_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Twenty_Day_Moving_Average_na'] = df['Twenty_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df = df.fillna('Three_Day_Moving_Average',99999)
    df = df.fillna('Five_Day_Moving_Average',99999)
    df = df.fillna('Ten_Day_Moving_Average',99999)
    df = df.fillna('Twenty_Day_Moving_Average',99999)
    df['MA_3_5']  = df['Three_Day_Moving_Average'] - df['Five_Day_Moving_Average']
    df['MA_3_10']  = df['Three_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_5_10']  = df['Five_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_20_10']  = df['Twenty_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_20_5']  = df['Twenty_Day_Moving_Average'] - df['Five_Day_Moving_Average']
    df['MA_20_3']  = df['Twenty_Day_Moving_Average'] - df['Three_Day_Moving_Average']
    ## releasing the vol features
    df['vol_3']    = df['Three_Day_Moving_Average'] - df['Volume']
    df['vol_5']    = df['Five_Day_Moving_Average'] - df['Volume']
    df['vol_10']    = df['Ten_Day_Moving_Average'] - df['Volume']
    df['vol_20']    = df['Twenty_Day_Moving_Average'] - df['Volume']
    df['Direction_add'] = df['Positive_Directional_Movement'] + df['Negative_Directional_Movement']
    df['Direction_sub'] = df['Positive_Directional_Movement'] - df['Negative_Directional_Movement']
    df['t_a']  = df['True_Range'] - df['Average_True_Range']
    # last hour feature'
    df['t_a_p']  = df['True_Range'] + df['Average_True_Range']
    df['MA_last_10_3'] = (df['Ten_Day_Moving_Average']*10 - df['Three_Day_Moving_Average']*3)/7
    df['MA_last_10_5'] = (df['Ten_Day_Moving_Average']*10 - df['Five_Day_Moving_Average']*5)/5
    df['MA_last_5_3'] = (df['Five_Day_Moving_Average']*5 - df['Three_Day_Moving_Average']*3)/2
    # last hour feature'
    #df['MA_std'] = df.apply(std_compute)
    return df
train = create_feature(train)
test = create_feature(test)

In [4]:
## exclude the unwanted features
features = test.column_names()
features_exclude = ['ID','timestamp','Stock_ID']
features1 = []

for f in features :
    if f not in features_exclude:
        features1.append(f)


In [5]:
## Split the train data into training data and validation data set
train_1,val_1 = train.random_split(0.8,seed=0)

In [10]:
## Display the features
features1

['Volume',
 'Three_Day_Moving_Average',
 'Five_Day_Moving_Average',
 'Ten_Day_Moving_Average',
 'Twenty_Day_Moving_Average',
 'True_Range',
 'Average_True_Range',
 'Positive_Directional_Movement',
 'Negative_Directional_Movement',
 'Three_Day_Moving_Average_na',
 'Five_Day_Moving_Average_na',
 'Ten_Day_Moving_Average_na',
 'Twenty_Day_Moving_Average_na',
 'MA_3_5',
 'MA_3_10',
 'MA_5_10',
 'MA_20_10',
 'MA_20_5',
 'MA_20_3',
 'vol_3',
 'vol_5',
 'vol_10',
 'vol_20',
 'Direction_add',
 'Direction_sub',
 't_a',
 't_a_p',
 'MA_last_10_3',
 'MA_last_10_5',
 'MA_last_5_3']

In [11]:
## Graphlab gradient boosting model. Is not used for the final submission
model = graphlab.boosted_trees_classifier.create(train_1,features=features1,target='Outcome',class_weights='auto',validation_set=None,row_subsample=0.8,column_subsample=0.8,max_depth=6,max_iterations=500,step_size=0.05, min_loss_reduction=4, min_child_weight=4,random_seed=0,verbose=True)

WARNING: Detected extremely low variance for feature(s) 'Three_Day_Moving_Average_na', 'Five_Day_Moving_Average_na', 'Ten_Day_Moving_Average_na', 'Twenty_Day_Moving_Average_na' because all entries are nearly the same.
Proceeding with model training using all features. If the model does not provide results of adequate quality, exclude the above mentioned feature(s) from the input dataset.

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 562316

Number of classes           : 2

...

| 485       | 223.923958   | 0.569140          | 0.674691          |

| 490       | 226.139389   | 0.569261          | 0.674635          |

| 495       | 228.461459   | 0.569387          | 0.674592          |

| 500       | 230.641698   | 0.569440          | 0.674567          |

+-----------+--------------+-------------------+-------------------+

In [12]:
## Graphlab gradient boosting model evaluation. Is not used for the final submission
model.evaluate(val_1)

{'accuracy': 0.5400753437827137,
 'auc': 0.5706768162845055,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        | 41175 |
 |      0       |        0        | 35983 |
 |      1       |        1        | 39856 |
 |      1       |        0        | 23409 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.5524200255031324,
 'log_loss': 0.6820540611775219,
 'precision': 0.4918611395638706,
 'recall': 0.6299849837983087,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+-------+
 | threshold | fpr | tpr |   p   |   n   |
 +-----------+-----+-----+-------+-------+
 |    0.0    | 1.0 | 1.0 | 63265 | 77158 |
 |   1e-05   | 1.0 | 1.0 | 63265 | 77158 |
 |   2e-0

In [13]:
## Graphlab gradient boosting model evaluation. Is not used for the final submission
outcome = model.predict(test,output_type='probability')
results = graphlab.SFrame()
results['ID'] = test['ID']
results['Outcome'] = outcome
results.save('3.csv')

In [14]:
## Graphlab gradient boosting model result display. Is not used for the final submission

results

ID,Outcome
1_554,0.560742735863
2_554,0.377099752426
3_554,0.519151806831
4_554,0.325537711382
6_554,0.531478226185
7_554,0.593524038792
9_554,0.572466909885
10_554,0.573926568031
11_554,0.539610385895
12_554,0.663826286793


In [15]:
## Features display
features1

['Volume',
 'Three_Day_Moving_Average',
 'Five_Day_Moving_Average',
 'Ten_Day_Moving_Average',
 'Twenty_Day_Moving_Average',
 'True_Range',
 'Average_True_Range',
 'Positive_Directional_Movement',
 'Negative_Directional_Movement',
 'Three_Day_Moving_Average_na',
 'Five_Day_Moving_Average_na',
 'Ten_Day_Moving_Average_na',
 'Twenty_Day_Moving_Average_na',
 'MA_3_5',
 'MA_3_10',
 'MA_5_10',
 'MA_20_10',
 'MA_20_5',
 'MA_20_3',
 'vol_3',
 'vol_5',
 'vol_10',
 'vol_20',
 'Direction_add',
 'Direction_sub',
 't_a',
 't_a_p',
 'MA_last_10_3',
 'MA_last_10_5',
 'MA_last_5_3']

In [16]:
## no of features 
len(features1)

30

In [6]:
## This keras model is not used for the final submission. Only the x_train,y_train,x_test,y_test prepared here are used 
## for the final xgboost model

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization



batch_size = 1000
epochs = 20

# the data, shuffled and split between train and test sets
x_train = train_1[features1].to_numpy()
y_train = train_1['Outcome'].to_numpy()
x_test = val_1[features1].to_numpy()
y_test = val_1['Outcome'].to_numpy()


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(30,)))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(300, activation='relu',name='feature_layer'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
#model.add(Dense(300, activation='relu'))
#model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Using TensorFlow backend.


562316 train samples
140423 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 300)               9300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
feature_layer (Dense)        (None, 300)               90300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_2 (Dense)              (None,

In [7]:
## This keras model is not used for the final submission. It tries to build feautures from the above keras model and then train a xgbooost 
## with that tries to train an xgboost. Was not used for final submission as results were not good.

from keras.models import Model
features_model = Model(inputs=model.input,outputs=model.get_layer('feature_layer').output)
features_model.compile(loss='mse', optimizer='adam')
F_train = features_model.predict([x_train], batch_size=128)
F_val   = features_model.predict([x_test], batch_size=128)

import xgboost as xgb

dTrain = xgb.DMatrix(F_train, label=y_train)
dVal   = xgb.DMatrix(F_val, label=y_test)

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.02, 
    'max_depth': 6,
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    #'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'colsample_bytree': 0.8,
    'min_child_weight': 4,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 700,  [(dTrain,'train'),((dVal,'val'))], 
                verbose_eval=10, early_stopping_rounds=20)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


[0]	train-logloss:0.692975	val-logloss:0.692964
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.691593	val-logloss:0.691482
[20]	train-logloss:0.690673	val-logloss:0.690482
[30]	train-logloss:0.690057	val-logloss:0.689799
[40]	train-logloss:0.689649	val-logloss:0.689337
[50]	train-logloss:0.689372	val-logloss:0.689015
[60]	train-logloss:0.689188	val-logloss:0.688795
[70]	train-logloss:0.689066	val-logloss:0.688642
[80]	train-logloss:0.688983	val-logloss:0.688535
[90]	train-logloss:0.688928	val-logloss:0.68846
[100]	train-logloss:0.688891	val-logloss:0.688407
[110]	train-logloss:0.688867	val-logloss:0.68837
[120]	train-logloss:0.688851	val-logloss:0.688343
[130]	train-logloss:0.68884	val-logloss:0.688324
[140]	train-logloss:0.688833	val-logloss:0.688309
[150]	train-logloss:0.688829	val-logloss:0.688299
[160]	train-logloss:0.688826	val-logloss:0.688291
[170]	train-loglo

In [13]:
## Not relevant to the final submission
x_test1 = test[features1].to_numpy() 

In [14]:
## Not used for the final submission. predicting on the keras model
predictions_valid = model.predict(x_test1.astype('float32'),batch_size=batch_size,verbose=2)
results1 = graphlab.SFrame()
results1['ID'] = test['ID']
results1['Outcome'] = predictions_valid
results1['Outcome'] = results1['Outcome'].apply(lambda x:x[0]) 
#results1['Outcome'] = 0.5*(results1['Outcome'] + results['Outcome'])
results1.save('submission_12.csv')

In [8]:
## This is the final model.  It used the x_train,y_train,x_test,y_test from In [6]
import xgboost as xgb

dTrain = xgb.DMatrix(x_train, label=y_train)
dVal   = xgb.DMatrix(x_test, label=y_test)

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.02, 
    'max_depth': 6,
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    #'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'colsample_bytree': 0.8,
    'min_child_weight': 4,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 700,  [(dTrain,'train'),((dVal,'val'))], 
                verbose_eval=10, early_stopping_rounds=20)


[0]	train-logloss:0.692693	val-logloss:0.692703
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.688959	val-logloss:0.689062
[20]	train-logloss:0.686231	val-logloss:0.686418
[30]	train-logloss:0.684309	val-logloss:0.684588
[40]	train-logloss:0.682767	val-logloss:0.683178
[50]	train-logloss:0.681643	val-logloss:0.682188
[60]	train-logloss:0.68067	val-logloss:0.68134
[70]	train-logloss:0.679885	val-logloss:0.680687
[80]	train-logloss:0.679197	val-logloss:0.680141
[90]	train-logloss:0.678657	val-logloss:0.679744
[100]	train-logloss:0.678176	val-logloss:0.679392
[110]	train-logloss:0.677754	val-logloss:0.679111
[120]	train-logloss:0.677372	val-logloss:0.678879
[130]	train-logloss:0.677028	val-logloss:0.678694
[140]	train-logloss:0.676722	val-logloss:0.678537
[150]	train-logloss:0.676436	val-logloss:0.678396
[160]	train-logloss:0.67618	val-logloss:0.678275
[170]	train-loglo

In [9]:
## predictions on the final model... 
F_test = test[features1].to_numpy() 
dTest = xgb.DMatrix(F_test.astype('float32'))
pred = bst.predict(dTest, ntree_limit=bst.best_ntree_limit)
results2 = graphlab.SFrame()
results2['ID'] = test['ID']
results2['Outcome'] = pred
results2.save('submission_xgb5.csv')



In [10]:
print (1)

1


In [15]:
## Not relevant to the final submission
results3 = graphlab.SFrame()
results3['ID'] = test['ID']
results3['Outcome'] = 0.5*(results2['Outcome'] + results1['Outcome'])
results2.save('submission_comb.csv')

In [11]:
print (1)

1


In [17]:
### This is just for clarity purpose .. Combining all the relevant steps together from the final solution and discarding the 
## code that is not required

import graphlab
import numpy as np

## Load the train and test data
train = graphlab.SFrame('~/Downloads/AV/train.csv')
test = graphlab.SFrame('~/Downloads/AV/test.csv')

## Feature engineering 
def std_compute(row):
    return np.std([np.int(row['Three_Day_Moving_Average']),np.int(row['Five_Day_Moving_Average']),np.int(row['Ten_Day_Moving_Average'])])
    
def create_feature(df):
    df['Three_Day_Moving_Average_na'] = df['Three_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Five_Day_Moving_Average_na'] = df['Five_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Ten_Day_Moving_Average_na'] = df['Ten_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df['Twenty_Day_Moving_Average_na'] = df['Twenty_Day_Moving_Average'].apply(lambda x:1 if x == None else 0)
    df = df.fillna('Three_Day_Moving_Average',99999)
    df = df.fillna('Five_Day_Moving_Average',99999)
    df = df.fillna('Ten_Day_Moving_Average',99999)
    df = df.fillna('Twenty_Day_Moving_Average',99999)
    df['MA_3_5']  = df['Three_Day_Moving_Average'] - df['Five_Day_Moving_Average']
    df['MA_3_10']  = df['Three_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_5_10']  = df['Five_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_20_10']  = df['Twenty_Day_Moving_Average'] - df['Ten_Day_Moving_Average']
    df['MA_20_5']  = df['Twenty_Day_Moving_Average'] - df['Five_Day_Moving_Average']
    df['MA_20_3']  = df['Twenty_Day_Moving_Average'] - df['Three_Day_Moving_Average']
    ## releasing the vol features
    df['vol_3']    = df['Three_Day_Moving_Average'] - df['Volume']
    df['vol_5']    = df['Five_Day_Moving_Average'] - df['Volume']
    df['vol_10']    = df['Ten_Day_Moving_Average'] - df['Volume']
    df['vol_20']    = df['Twenty_Day_Moving_Average'] - df['Volume']
    df['Direction_add'] = df['Positive_Directional_Movement'] + df['Negative_Directional_Movement']
    df['Direction_sub'] = df['Positive_Directional_Movement'] - df['Negative_Directional_Movement']
    df['t_a']  = df['True_Range'] - df['Average_True_Range']
    # last hour feature'
    df['t_a_p']  = df['True_Range'] + df['Average_True_Range']
    df['MA_last_10_3'] = (df['Ten_Day_Moving_Average']*10 - df['Three_Day_Moving_Average']*3)/7
    df['MA_last_10_5'] = (df['Ten_Day_Moving_Average']*10 - df['Five_Day_Moving_Average']*5)/5
    df['MA_last_5_3'] = (df['Five_Day_Moving_Average']*5 - df['Three_Day_Moving_Average']*3)/2
    # last hour feature'
    #df['MA_std'] = df.apply(std_compute)
    return df
train = create_feature(train)
test = create_feature(test)


## exclude the unwanted features
features = test.column_names()
features_exclude = ['ID','timestamp','Stock_ID']
features1 = []

for f in features :
    if f not in features_exclude:
        features1.append(f)
        
x_train = train_1[features1].to_numpy()
y_train = train_1['Outcome'].to_numpy()
x_test = val_1[features1].to_numpy()
y_test = val_1['Outcome'].to_numpy()

import xgboost as xgb

dTrain = xgb.DMatrix(x_train, label=y_train)
dVal   = xgb.DMatrix(x_test, label=y_test)

xgb_params = {
    'objective': 'binary:logistic',
    'booster': 'gbtree',
    'eval_metric': 'logloss',
    'eta': 0.02, 
    'max_depth': 6,
    'lambda': 2,
    'alpha': 0.02,
    'subsample': 0.8,
    #'colsample_bytree': 1 / F_train.shape[1]**0.5,
    'colsample_bytree': 0.8,
    'min_child_weight': 4,
    'silent': 1
}
bst = xgb.train(xgb_params, dTrain, 700,  [(dTrain,'train'),((dVal,'val'))], 
                verbose_eval=10, early_stopping_rounds=20)


## predictions on the final model... 
F_test = test[features1].to_numpy() 
dTest = xgb.DMatrix(F_test.astype('float32'))
pred = bst.predict(dTest, ntree_limit=bst.best_ntree_limit)
results2 = graphlab.SFrame()
results2['ID'] = test['ID']
results2['Outcome'] = pred
results2.save('submission_xgb5.csv')


Finished parsing file /home/santanu/Downloads/AV/train.csv

Parsing completed. Parsed 100 lines in 0.558947 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,float,float,float,float,float,float,float,float,float,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 546558 lines. Lines per second: 586650

Finished parsing file /home/santanu/Downloads/AV/train.csv

...

Parsing completed. Parsed 100 lines in 0.110088 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/santanu/Downloads/AV/test.csv

Parsing completed. Parsed 101946 lines in 0.165098 secs.

[0]	train-logloss:0.692693	val-logloss:0.692703
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 20 rounds.
[10]	train-logloss:0.688959	val-logloss:0.689062
[20]	train-logloss:0.686231	val-logloss:0.686418
[30]	train-logloss:0.684309	val-logloss:0.684588
[40]	train-logloss:0.682767	val-logloss:0.683178
[50]	train-logloss:0.681643	val-logloss:0.682188
[60]	train-logloss:0.68067	val-logloss:0.68134
[70]	train-logloss:0.679885	val-logloss:0.680687
[80]	train-logloss:0.679197	val-logloss:0.680141
[90]	train-logloss:0.678657	val-logloss:0.679744
[100]	train-logloss:0.678176	val-logloss:0.679392
[110]	train-logloss:0.677754	val-logloss:0.679111
[120]	train-logloss:0.677372	val-logloss:0.678879
[130]	train-logloss:0.677028	val-logloss:0.678694
[140]	train-logloss:0.676722	val-logloss:0.678537
[150]	train-logloss:0.676436	val-logloss:0.678396
[160]	train-logloss:0.67618	val-logloss:0.678275
[170]	train-loglo